In [5]:
import glob
import os


from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords

import pandas as pd

from PIL import Image 
import matplotlib.pylab as plt

In [8]:
import os
import re
import time
from pathlib import Path
import shutil
import glob
import string
import unicodedata
from datetime import datetime, timedelta

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from sqlalchemy import create_engine
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from unicodedata import normalize, category

In [23]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

In [28]:
class ApplyTextRank(object):
    
    def __init__(self, hoy_format=None,  metodo=1):
        #-------------------------- Habilitacion de ruta --------------------------#
        self.folder_news = 'news'
        if not os.path.exists(self.folder_news):
            os.makedirs(self.folder_news)
        
        self.hoy = datetime.today()
        self.hoy_format = self.hoy.strftime('%d%m%Y') if hoy_format is None else hoy_format
        
        self.newsletter_path = os.path.join(self.folder_news, self.hoy_format, 'parseado')
        
        path_ = os.path.join(self.newsletter_path, 'metodo_{}.txt'.format(metodo))
        with open(path_, 'r', encoding="utf-8") as file:
            parseado = file.read().replace('\n', '.')
            
        self._sentences = [oracion.replace('.', '') for oracion in sent_tokenize(parseado)]
        self._stopwords = stopwords.words('spanish')

        
    def remove_stopwords(self):
        self._sentences = [
            [words for words in sentence.split(' ') if words not in self._stopwords] 
            for sentence in self._sentences
        ]
        

In [34]:
apply_text_rank = ApplyTextRank()
apply_text_rank.remove_stopwords()
#apply_text_rank._sentences

In [35]:
w2v = Word2Vec(
    apply_text_rank._sentences, 
    size=1,
    min_count=1,
    iter=1000
)

In [37]:
sentence_embeddings = [
    [w2v[word][0] for word in words] 
    for words in apply_text_rank._sentences
]

<ipython-input-37-516a21e910ff>:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  [w2v[word][0] for word in words]


In [38]:
max_len =max(
    [len(tokens) for tokens in apply_text_rank._sentences]
)
max_len

417

In [39]:
sentence_embeddings = [
    np.pad(embedding,(0,max_len-len(embedding)),'constant') 
    for embedding in sentence_embeddings
]

In [42]:
similarity_matrix = np.zeros([len(apply_text_rank._sentences), len(apply_text_rank._sentences)])
similarity_matrix.shape

(15, 15)

In [43]:
for i, row_embedding in enumerate(sentence_embeddings):
    for j, column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j] = 1 - spatial.distance.cosine(row_embedding, column_embedding)

### PAGE RANK

In [44]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [45]:
scores

{0: 0.06831668601221846,
 1: 0.07028714988740042,
 2: 0.058081250660500156,
 3: 0.06215739469081619,
 4: 0.06530097658879047,
 5: 0.06776864347324314,
 6: 0.06774956393315905,
 7: 0.07043381072422315,
 8: 0.06378356776319019,
 9: 0.058846710303357856,
 10: 0.07078004667285599,
 11: 0.07144961215364169,
 12: 0.0714407069831776,
 13: 0.06531997718251457,
 14: 0.06828390297091104}

### TOP

In [55]:
top_sentence={
    sentence[:scores[index] for index, sentence in list(enumerate(apply_text_rank._sentences))
}

In [56]:
top_sentence

{'pandemia': 0.06831668601221846,
 'autoridad': 0.07028714988740042,
 'personas': 0.058081250660500156,
 'direccion': 0.06215739469081619,
 'presidenta': 0.06530097658879047,
 'ministerio': 0.06531997718251457,
 'vuelta': 0.07043381072422315,
 'menos': 0.06378356776319019,
 'suponia': 0.058846710303357856,
 'ministro': 0.06828390297091104}